# Description
This note book is using Google earth engine, Arcpy, ArcGIS pro to exporting jpg files.

In [ ]:
# Using the arcgis pro notebook to run the code
# try import jpg from arcgis

import arcpy

# return selected count of feature class
arcpy.GetCount_management("1823_ADRSM") # if selected 1 it will return 1


In [ ]:
# add gee into arcgispro

import ee
import geemap


# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-qinheyi')

In [ ]:
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

m=geemap.Map()


def getDateRange(imageCollection:ee.ImageCollection):
    dateRange=imageCollection.aggregate_array('system:time_start')
    date_list=[ee.Date(date).format().getInfo() for date in dateRange.getInfo()]
    print(date_list)
    return date_list

# imageRange=getDateRange(nicfi) # because of the server-side function.

baseMap=nicfi.filter(ee.Filter.date('2024-01-01','2025-01-11')).sort('system:time_start', False).first()

# m.centerObject(baseMap,4)

vis={'bands':['R','G','B'],'min':64,'max':5454,'gamma':1.8}
m.addLayer(baseMap,vis,'nicfi')
m.height='800px'

display(m)

In [ ]:
#get the project and layout

current_project=arcpy.mp.ArcGISProject("CURRENT")
layout=current_project.listLayouts()[1]  # 0 is the dataframe drive layout(text,area) # 1 is the 
print("current layout name:",layout.name)


In [ ]:
# get the certain shape of shapefile  and iterate the shapefile
# handl the mv , do not effect the layout 
import time 
start_time = time.time()

fc = "1823_ADRSM"
fields = ['DEN_BOT','AREA_M2','Index',"SHAPE@" ]

#get the mapframe in the layout
mf= layout.listElements("MAPFRAME_ELEMENT")[0]
path=r"C:\Users\lycaz\Desktop\JPEGS_Index2"

# get he m, mv
amap=current_project.listMaps()[0]
mv=amap.defaultView


with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        #print(u'{0}, {1},{2}'.format(row[2],row[0], row[1]))
        print(f"Processing index {row[2]} {row[0]}", end="\r")
        #if row[2] == 1124:
        geometry=row[3]
        mv.camera.setExtent(geometry.extent)
        amap.defaultCamera=mv.camera
        amap.defaultCamera.scale*=1.5
        mf.camera.setExtent(amap.defaultCamera.getExtent())
        mf.camera.scale*=1.3
        layout.exportToJPEG(f"{path}\\{str(row[2])}.jpg",resolution=300)
    print('\n')
    print("Complete all the image")

stop_time = time.time()
elapsed_time = stop_time - start_time

print(f"Start time: {time.ctime(start_time)}")
print(f"Stop time: {time.ctime(stop_time)}")
print(f"Elapsed time: {elapsed_time:.2f} seconds")